Test loading modules
======================

In [1]:
using Revise
using Clang, Clang_jll

In [2]:
## for attaching lldb
getpid()

88299

In [2]:
## check custom libclang is loaded from LocalPreferences
Clang_jll.libclang

"/Users/nehal/src/exploration/llvm-project/llvm/cmake-build-debug/lib/libclang.dylib"

In [3]:
sdk_dir = `xcrun --show-sdk-path` |> open |> readchomp |> String

"/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk"

In [4]:
index = Index()

Index(Ptr{Nothing} @0x000000029d9c9b90, 0, 1)

Load without modules, using umbrella header
===========================================

In [5]:
args = [
    "-x","objective-c",
    "-isysroot", sdk_dir,
    # "-fmodules", 
    # "-fmodules-cache-path=cache",
    # "-fimplicit-module-maps",
    # #"-fmodules-validate-system-headers", 
    "-fblocks", 
    "-fregister-global-dtors-with-atexit",
    "-fgnuc-version=4.2.1", 
    "-fobjc-runtime=macosx-13.0.0", 
    "-fobjc-exceptions", 
    "-fexceptions",
    "-fmax-type-align=16",
    "-fcommon", 
    "-resource-dir",
    "/Users/nehal/.julia/artifacts/c9828f0ef0f967487ce376a3d8b4c8a646d38ace/lib/clang/13.0.1",
    "-DNS_FORMAT_ARGUMENT(A)=",
    "-D__GCC_HAVE_DWARF2_CFI_ASM=1",
    "-v",
    ]

    """
    A bit of explanation on what is going on.
    In Xcode 13 RC localizedAttributedStringForKey:value:table: method got NS_FORMAT_ARGUMENT(1) macro attribute. Previously, this attribute was applicable only to functions that return a C string, CFString or NSString. Recently, Apple added support in Clang for another type, NSAttributedString. Clang that ships with Kotlin/Native does not have this patch and fails to process localizedAttributedStringForKey:value:table: declaration.
    What this workaround does is it makes NS_FORMAT_ARGUMENT(1) a no-op. It is perfectly fine in this case since cinterop does not take this attribute into account anyway.
    https://github.com/apple/llvm-project/commit/2de0a18a8949f0235fb3a08dcc55ff3aa7d969e7
    
    """

trans_unit = Clang.parse_header(index, "050_import.h", args)

clang version 13.0.1 (https://github.com/habemus-papadum/llvm-project.git 75e33f71c2dae584b13a7d1186ae0a038ba98838)
Target: arm64-apple-darwin22.1.0
Thread model: posix
InstalledDir: 
ignoring nonexistent directory "/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/local/include"
ignoring nonexistent directory "/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/Library/Frameworks"
#include "..." search starts here:
#include <...> search starts here:
 /Users/nehal/.julia/artifacts/c9828f0ef0f967487ce376a3d8b4c8a646d38ace/lib/clang/13.0.1/include
 /Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include
 /Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/System/Library/Frameworks (framework directory)
End of search list.


TranslationUnit(Ptr{Clang.LibClang.CXTranslationUnitImpl} @0x000000011ff98df0, Index(Ptr{Nothing} @0x000000029d9c9b90, 0, 1))

In [6]:
trans_unit |> Clang.getTranslationUnitCursor |> children

15539-element Vector{CLCursor}:
 CLCursor (CLObjCClassRef) Protocol
 CLCursor (CLTypedefDecl) __int8_t
 CLCursor (CLTypedefDecl) __uint8_t
 CLCursor (CLTypedefDecl) __int16_t
 CLCursor (CLTypedefDecl) __uint16_t
 CLCursor (CLTypedefDecl) __int32_t
 CLCursor (CLTypedefDecl) __uint32_t
 CLCursor (CLTypedefDecl) __int64_t
 CLCursor (CLTypedefDecl) __uint64_t
 CLCursor (CLTypedefDecl) __darwin_intptr_t
 ⋮
 CLCursor (CLObjCClassRef) NSXPCConnection
 CLCursor (CLObjCInterfaceDecl) NSUserScriptTask
 CLCursor (CLTypedefDecl) NSUserScriptTaskCompletionHandler
 CLCursor (CLObjCInterfaceDecl) NSUserUnixTask
 CLCursor (CLTypedefDecl) NSUserUnixTaskCompletionHandler
 CLCursor (CLObjCInterfaceDecl) NSUserAppleScriptTask
 CLCursor (CLTypedefDecl) NSUserAppleScriptTaskCompletionHandler
 CLCursor (CLObjCInterfaceDecl) NSUserAutomatorTask
 CLCursor (CLTypedefDecl) NSUserAutomatorTaskCompletionHandler

Load with modules from Libclang
=================================
Fails, TODO debug why?

In [7]:
args = [
    "-x","objective-c",
    "-isysroot", sdk_dir,
    "-fmodules", 
    "-fmodules-cache-path=cache",
    "-fimplicit-module-maps",
    "-fmodules-validate-system-headers", 
    "-fblocks", 
    "-fregister-global-dtors-with-atexit",
    "-fgnuc-version=4.2.1", 
    "-fobjc-runtime=macosx-13.0.0", 
    "-fobjc-exceptions", 
    "-fexceptions",
    "-fmax-type-align=16",
    "-fcommon", 
    "-resource-dir",
    "/Users/nehal/.julia/artifacts/c9828f0ef0f967487ce376a3d8b4c8a646d38ace/lib/clang/13.0.1",
    "-DNS_FORMAT_ARGUMENT(A)=",
    "-D__GCC_HAVE_DWARF2_CFI_ASM=1",
    "-v",
    ]

    """
    A bit of explanation on what is going on.
    In Xcode 13 RC localizedAttributedStringForKey:value:table: method got NS_FORMAT_ARGUMENT(1) macro attribute. Previously, this attribute was applicable only to functions that return a C string, CFString or NSString. Recently, Apple added support in Clang for another type, NSAttributedString. Clang that ships with Kotlin/Native does not have this patch and fails to process localizedAttributedStringForKey:value:table: declaration.
    What this workaround does is it makes NS_FORMAT_ARGUMENT(1) a no-op. It is perfectly fine in this case since cinterop does not take this attribute into account anyway.
    https://github.com/apple/llvm-project/commit/2de0a18a8949f0235fb3a08dcc55ff3aa7d969e7
    
    """

trans_unit = Clang.parse_header(index, "050_import.h", args)

clang version 13.0.1 (https://github.com/habemus-papadum/llvm-project.git 75e33f71c2dae584b13a7d1186ae0a038ba98838)
Target: arm64-apple-darwin22.1.0
Thread model: posix
InstalledDir: 
ignoring nonexistent directory "/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/local/include"
ignoring nonexistent directory "/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/Library/Frameworks"
#include "..." search starts here:
#include <...> search starts here:
 /Users/nehal/.julia/artifacts/c9828f0ef0f967487ce376a3d8b4c8a646d38ace/lib/clang/13.0.1/include
 /Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include
 /Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/System/Library/Frameworks (framework directory)
End of search list.
clang -cc1 version 13.0.1 based upon LLVM 13.0.1 default target arm64-apple-darwin22

TranslationUnit(Ptr{Clang.LibClang.CXTranslationUnitImpl} @0x000000029dbbd460, Index(Ptr{Nothing} @0x000000029d9c9b90, 0, 1))

Precompiled Headers
=======================
- Try creating pch via clang and then loading via libclang (to work around module building problems) 
   

## Digression
---------------
Using even slightly mismatched clang and libclang cuases problems (e.g. same version but different git describe)

In [8]:
run(`$(clang()) -x objective-c 050_import.h -emit-ast -o test.pch -fmodules -isysroot $sdk_dir "-DNS_FORMAT_ARGUMENT(A)="`);
nothing
#clang -x objective-c 050_import.h -emit-ast -o test.pch -fmodules

In [9]:
ptr = Clang.clang_createTranslationUnit(index, "test.pch")
## fails -- Using a custom built libclang but jll clang has version mismatch due to git hash being different -- need to lldb to see this, there is flag that disables this
## level of version compatibility checking but not sure how to pass it to libclang

Ptr{Clang.LibClang.CXTranslationUnitImpl} @0x0000000000000000

## Try clang from same build as libclang
First using a non-module header from last time

In [10]:
myclang = "/Users/nehal/src/exploration/llvm-project/llvm/cmake-build-debug/bin/clang"
run(`$(myclang) -x c 048_testa.h -emit-ast -o testa.pch -isysroot $sdk_dir`);

In [11]:
ptr = Clang.clang_createTranslationUnit(index, "testa.pch")

Ptr{Clang.LibClang.CXTranslationUnitImpl} @0x000000011fd17110

In [12]:
## works!

In [13]:
tu = load_ast(index, "testa.pch")

TranslationUnit(Ptr{Clang.LibClang.CXTranslationUnitImpl} @0x000000029dcd1660, Index(Ptr{Nothing} @0x000000029d9c9b90, 0, 1))

In [14]:
for c in tu |> Clang.getTranslationUnitCursor |> children
    println("$(c)\t$(get_filename(c))")
end

CLCursor (CLFunctionDecl) c()	/Users/nehal/src/ObjectiveC.jl/exploration/048_testc.h
CLCursor (CLFunctionDecl) b()	/Users/nehal/src/ObjectiveC.jl/exploration/048_testb.h
CLCursor (CLStructDecl) 	/Users/nehal/src/ObjectiveC.jl/exploration/048_testa.h


CLCursor (CLTypedefDecl) Bar	/Users/nehal/src/ObjectiveC.jl/exploration/048_testa.h


Now Back to focusing on Modules
================================

PCH with Modules, using legacy `#import`
======================

In [15]:
# pch with modules, #import
run(`$(myclang) -x objective-c 050_import.h -emit-ast -o import.pch -fmodules -isysroot $sdk_dir "-DNS_FORMAT_ARGUMENT(A)="`);

In [16]:
for c in load_ast(index, "import.pch") |> Clang.getTranslationUnitCursor |> children |> x -> Iterators.take(x,10)
    println("$(c)\t$(get_filename(c))")
end

CLCursor (CLObjCClassRef) Protocol	
CLCursor (CLTypedefDecl) max_align_t	/Users/nehal/src/exploration/llvm-project/llvm/cmake-build-debug/lib/clang/13.0.1/include/__stddef_max_align_t.h
CLCursor (CLFunctionDecl) cacosf(_Complex float)	/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/complex.h
CLCursor (CLFunctionDecl) cacos(_Complex double)	/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/complex.h
CLCursor (CLFunctionDecl) cacosl(_Complex long double)	/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/complex.h
CLCursor (CLFunctionDecl) casinf(_Complex float)	/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/complex.h
CLCursor (CLFunctionDecl) casin(_Complex double)	/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/

PCH with Modules, using `@import`
======================

In [17]:
# pch with modules, @import
run(`$(myclang) -x objective-c 050_import_module.h -emit-ast -o import_module.pch -fmodules -isysroot $sdk_dir "-DNS_FORMAT_ARGUMENT(A)="`);

In [18]:
for c in load_ast(index, "import_module.pch") |> Clang.getTranslationUnitCursor |> children |> x -> Iterators.take(x,10)
    println("$(c)\t$(get_filename(c))")
end

CLCursor (CLObjCClassRef) Protocol	
CLCursor (CLTypedefDecl) max_align_t	/Users/nehal/src/exploration/llvm-project/llvm/cmake-build-debug/lib/clang/13.0.1/include/__stddef_max_align_t.h
CLCursor (CLFunctionDecl) cacosf(_Complex float)	/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/complex.h
CLCursor (CLFunctionDecl) cacos(_Complex double)	/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/complex.h
CLCursor (CLFunctionDecl) cacosl(_Complex long double)	/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/complex.h
CLCursor (CLFunctionDecl) casinf(_Complex float)	/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/complex.h
CLCursor (CLFunctionDecl) casin(_Complex double)	/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/

In [19]:
tu = load_ast(index, "import_module.pch")
toplevel =  tu |> Clang.getTranslationUnitCursor |> children |> collect 
cu = toplevel[3]


CLCursor (CLFunctionDecl) cacosf(_Complex float)

Module Names
=======================

In [21]:
## modules
file = Ref{Clang.CXFile}(C_NULL)
location = Clang.clang_getCursorLocation(cu)
Clang.clang_getExpansionLocation(location, file, Ref{Cuint}(0), Ref{Cuint}(0), Ref{Cuint}(0))
mptr = Clang.clang_getModuleForFile(tu, file[])
cxstr = Clang.clang_Module_getFullName(mptr)
ptr = Clang.clang_getCString(cxstr)
s = ptr == C_NULL ? "" : unsafe_string(ptr)
Clang.clang_disposeString(cxstr)
s

ErrorException: cannot assign a value to variable Clang.file from module Main

In [20]:
cu2 = toplevel[3]
file2 = Ref{Clang.CXFile}(C_NULL)
location2 = Clang.clang_getCursorLocation(cu2)
Clang.clang_getExpansionLocation(location2, file2, Ref{Cuint}(0), Ref{Cuint}(0), Ref{Cuint}(0))
mptr2 = Clang.clang_getModuleForFile(tu, file2[])
@show file, file2
@show mptr, mptr2

(file, file2) = (Clang.file, Base.RefValue{Ptr{Nothing}}(Ptr{Nothing} @0x0000000102e2a0e0))


UndefVarError: UndefVarError: mptr not defined

In [25]:
Clang.version()

"clang version 13.0.1 (https://github.com/habemus-papadum/llvm-project.git 75e33f71c2dae584b13a7d1186ae0a038ba98838)"

In [12]:
Base.libllvm_version

v"13.0.1"